In [2]:
import requests
from bs4 import BeautifulSoup
import csv

links = list()

url = 'https://movies.yahoo.com.tw/movie_thisweek.html'
r = requests.get(url)
soup = BeautifulSoup(r.text)

with open('NLP_HW02.csv', 'w', newline='', encoding='UTF-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['名稱', '分類', '劇情介紹', '上映日期', '網址'])
with open('NLP_ACTLINK_HW02.csv', 'w', newline='', encoding='UTF-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['網址'])

for link in soup.find_all('div', class_="release_movie_name"):
    links.append(link.find('a', class_='gabtn')['href'])

def movielink(movie):
    exist=0
    #檢查該電影是否查找過
    print(movie)
    if(movie.find('a',class_="gabtn")):
      with open('NLP_HW02.csv', newline='', encoding='utf-8') as csvfile:
          rows = csv.DictReader(csvfile)
          for row in rows:
              if(row['網址']==movie.find('a',class_="gabtn")['href']):
                  exist=1
                  break;
          #如果沒有查找過，進去該連結
          if(exist==0):
              movieinfo(movie.find('a',class_="gabtn")['href'])
    
def moveact(act):
    if(act.find_all('a',class_="gabtn")):
        for link in act.find_all('a',class_="gabtn"):
            change=0
            #抓出演員的連結並檢查是否重複
            with open('NLP_ACTLINK_HW02.csv', newline='', encoding='utf-8') as csvfile:
                    rows = csv.DictReader(csvfile)
                    for row in rows:
                        if(row['網址']==link['href']):
                            change=1;
                            break;
            #如果沒有重複
            if(change==0):
                #寫入檔案
                with open('NLP_ACTLINK_HW02.csv', 'a', newline='', encoding='UTF-8') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow([link['href']])
                url = link['href']
                r = requests.get(url)
                Soup = BeautifulSoup(r.text)
                #抓出該演員演過的所有電影連結
                for movie in Soup.find_all('div',class_="_slickcontent"):
                    movielink(movie)
    
def movieinfo(link):
    url = link
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    Class=''
    
    #抓電影名稱
    moviename=soup.find('div',class_="movie_intro_info_r").find('h1').text
    #抓電影介紹
    movieintro=soup.find('div',class_="gray_infobox_inner").find('span').text
    #抓上映日期
    update=soup.find('div',class_="movie_intro_info_r").find('span').text
    #抓分類
    for mark in soup.find_all('div',class_="level_name"):
        if(mark.find('a',class_="gabtn")):
            if(mark.find('a',class_="gabtn").text):
                Class=mark.find('a',class_="gabtn").text.strip()+" "+Class
            else:
                Class="NaN"
    #寫檔
    with open('NLP_HW02.csv', 'a', newline='', encoding='UTF-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([moviename, Class, movieintro, update, link])
    for act in soup.find_all('div',class_="movie_intro_list"):
        moveact(act)
    
    
#抓最一開始時所有電影的連結
for link in links:
    movieinfo(link)

KeyboardInterrupt: 

In [3]:
import sys
sys.setrecursionlimit(1000000)